In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint


In [3]:
from scipy import ndimage

In [4]:
import os,shutil,glob

In [6]:
imgfiles = [f for f in glob.glob("./data/*.jpg")]

In [19]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image 

In [20]:
from tqdm.notebook import trange, tqdm

In [21]:
for filename in tqdm(imgfiles):
    image_file = Image.open(filename) # open colour image
    
    image_file = image_file.convert('L') # convert image to black and white
    image_file.save(filename)

In [31]:
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [226]:
X = []
Y = []

In [227]:
for imgfile in imgfiles:
#     img = mpimg.imread(imgfile)
#     print(img.shape)
#     print(img.thumbnail((150, 150)))
    image = mpimg.imread(imgfile)
    image_resized = resize(image, (150, 150),
                       anti_aliasing=True)
#     print(np.shape(list(np.resize(image_resized,(150,150,1)))))
    X.append(list(np.resize(image_resized,(150,150,1))))
    if(imgfile.find("y") != -1):
#         print(imgfile)
        Y.append(1)
    if(imgfile.find("n") != -1):
        Y.append(0)
            
#     print(img[:,:,0].shape)

In [240]:
np.shape(X)

(178, 150, 150, 1)

In [229]:
np.shape(Y)

(178,)

In [89]:
from sklearn.model_selection import train_test_split


In [230]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    shuffle=True,
    random_state=42,
)

In [238]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [239]:
(X_train.shape)

(142, 150, 150, 1)

In [95]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [243]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [244]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [245]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [246]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)        36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 73984)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

In [247]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [248]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))

Train on 142 samples, validate on 36 samples
Epoch 1/10
142/142 [==============================] - 3s 18ms/sample - loss: 0.7400 - accuracy: 0.5211 - val_loss: 0.5532 - val_accuracy: 0.8056
Epoch 2/10
142/142 [==============================] - 2s 15ms/sample - loss: 0.5428 - accuracy: 0.7887 - val_loss: 0.4971 - val_accuracy: 0.8056
Epoch 3/10
142/142 [==============================] - 2s 15ms/sample - loss: 0.4427 - accuracy: 0.7887 - val_loss: 0.4124 - val_accuracy: 0.8333
Epoch 4/10
142/142 [==============================] - 2s 16ms/sample - loss: 0.3290 - accuracy: 0.8662 - val_loss: 0.3636 - val_accuracy: 0.8611
Epoch 5/10
142/142 [==============================] - 2s 16ms/sample - loss: 0.2498 - accuracy: 0.9085 - val_loss: 0.3641 - val_accuracy: 0.8889
Epoch 6/10
142/142 [==============================] - 2s 17ms/sample - loss: 0.2117 - accuracy: 0.9225 - val_loss: 0.3516 - val_accuracy: 0.8889
Epoch 7/10
142/142 [==============================] - 2s 16ms/sample - loss: 0.1804 -

In [251]:

# X_test

y_test

array([1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1])

In [252]:
model.predict([X_test[0:5]])

array([[0.00126705, 0.998733  ],
       [0.28198397, 0.71801597],
       [0.9986222 , 0.00137787],
       [0.9160506 , 0.08394941],
       [0.30180368, 0.6981963 ]], dtype=float32)

In [253]:
model.save("btumor_model.h5")

In [260]:
X_test[0:1].shape

(1, 150, 150, 1)

In [262]:
X_test[0].shape

(150, 150, 1)